In [ ]:
import gym
import numpy as np
import numpy.matlib as npmat
import matplotlib.pyplot as plt
from collections import deque
import tensorflow as tf
import random
import threading

def toggle_rendering():

    global render
    global runflag
    while runflag:
        p = raw_input()
        if p == 'q':
            runflag = False
        else:
            render = not render
            
            
            
class Q_agente_DNN(object):   
    
    def __init__(self,
                 dim_input,      
                 dim_hidden1,    
                 dim_hidden2,    
                 n_accao,        
                 gamma,
                 epsilon,
                 learning_rate,  
                 regularisation_l=0.0,
                 momentum=0.2):
        
        
        # Sessão de serviço para correr o grafo
        self.sess = tf.Session()
        
        # Dimensões da rede
        self.n_input    = dim_input      #dimensão do input
        self.n_hidden_1 = dim_hidden1    #dimensão da primeira layer
        self.n_hidden_2 = dim_hidden2    #dimensão da segunda layer
        self.n_output   = n_accao        #número de acções, igual à dimensão do output da rede
       
        # Parâmetros de Aprendizagem, relacionados com o treino ou a função de custo
        self.learning_rate   = learning_rate                #learning rate do optimizador 
        self.gamma           = gamma                        #desconto de rewards futuras
        self.epsilon         = epsilon                      #probabilidade de escolher uma acção random
        
        # Constante do regularizador da função objectiva
        self.reg = 0.01  
        
        # Variáveis de percepcção e actuação do agente, inclui observação, acção e reward correspondente
        self.x = tf.placeholder("float", [None, self.n_input])  # Perfil do tensor de observação, contêm a dimensão do input
        self.r = tf.placeholder("float", [None, 1])             # Reward obtida
        self.a = tf.placeholder("float", [None, n_accao])       # Acção exercida pelo agente 
        self.future_rewards = tf.placeholder("float", [None])   # No futuro usar tf.Variable_Scope para lidar de forma mais segura com tf.Variables
        print ("Q agente iniciado")
        
        # Dicionários com os pesos e enviasamento da rede
        self.stddev = 0.1
        self.pesos = {
            'h1': tf.Variable(tf.random_normal([self.n_input, self.n_hidden_1], stddev=self.stddev)),
            'h2': tf.Variable(tf.random_normal([self.n_hidden_1, self.n_hidden_2], stddev=self.stddev)),
            'out': tf.Variable(tf.random_normal([self.n_hidden_2, self.n_output], stddev=self.stddev))
        }
        self.enviasamentos = {
            'b1': tf.Variable(tf.random_normal([self.n_hidden_1],stddev=self.stddev)),
            'b2': tf.Variable(tf.random_normal([self.n_hidden_2],stddev=self.stddev)),
            'out': tf.Variable(tf.random_normal([self.n_output],stddev=self.stddev))
        }
    
        # Rede Q: dim_input---»dim_h(RELU)---»dim_h(RELU)---»dim_output(LINEAR)
    def Q_rede(self,_X, _pesos, _enviasamentos):
        layer_1 = tf.nn.relu(tf.add(tf.matmul(_X, _pesos['h1']), _enviasamentos['b1'])) 
        layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, _pesos['h2']), _enviasamentos['b2']))
        output  = tf.matmul(layer_2, _pesos['out']) + _enviasamentos['out']
        return output

    
    def Q_agente_initializar(self):
        # Definir função objectiva, optimizador
       
        self.pred = self.Q_rede(self.x, self.pesos, self.enviasamentos)
        self.y  = tf.reduce_sum(tf.mul(self.pred,self.a), reduction_indices = 1)
        
        # Se quisermos incluir regularização, embora o batch aleatório de samples funcione como regularizador
        #weightL2loss = self.reg*tf.nn.l2_loss(self.pesos['h1']) + self.reg*tf.nn.l2_loss(self.pesos['h2']) + self.reg*tf.nn.l2_loss(self.pesos['out'])
        #biasL2loss = self.reg*tf.nn.l2_loss(self.enviasamentos['b1']) + self.reg*tf.nn.l2_loss(self.enviasamentos['b1']) + self.reg*tf.nn.l2_loss(self.enviasamentos['out'])
        #self.future_rewards = tf.add(self.r,tf.mul(self.gamma,self.q_target))
        
        # Função objectiva e optimizador
        self.cost = tf.reduce_mean(tf.square(self.future_rewards - self.y)) #+ weightL2loss + biasL2loss
        #self.cost = tf.reduce_mean(tf.square(self.future_rewards - self.y)) + weightL2loss + biasL2loss#+ weightL2loss + biasL2loss
        self.optm = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.cost) # Adam Optimizer

        # Initializar todas as variáveis (i.e, ligar e initializar os nós do grafo) e a sessão de trabalho
        self.init = tf.initialize_all_variables()
        self.sess.run(self.init)
        
    
    def actua(self,obs):
        
        accao = np.argmax(self.sess.run(self.pred, feed_dict={self.x: obs}))
        
        return accao

        
    
    # Classe da target Network, herda a Q_network e obtêm mais um método para copiar pesos. Foi criada em separado para existir numa sessão completamente
    # diferente (conflictos de variáveis)
class Target_Q_agente_DNN(Q_agente_DNN):
    
    def __init__(self, 
                 dim_input,      
                 dim_hidden1,    
                 dim_hidden2,    
                 n_accao,        
                 gamma,
                 epsilon,
                 learning_rate,  
                 regularisation_l=0.0,
                 momentum=0.2):
        super(Target_Q_agente_DNN, self).__init__(dim_input,      
                 dim_hidden1,    
                 dim_hidden2,    
                 n_accao,        
                 gamma,
                 epsilon,
                 learning_rate,  
                 regularisation_l,
                 momentum)
        
    # Copiar pesos de outra sessão para esta
    def copiar_pesos(self,q_network):
        
        self.op1=self.pesos['h1'].assign(q_network.pesos['h1'].eval(session = q_network.sess))
        self.op2=self.pesos['h2'].assign(q_network.pesos['h2'].eval(session = q_network.sess))
        self.op3=self.pesos['out'].assign(q_network.pesos['out'].eval(session = q_network.sess))
        
        self.op4=self.enviasamentos['b1'].assign(q_network.enviasamentos['b1'].eval(session = q_network.sess))
        self.op5=self.enviasamentos['b2'].assign(q_network.enviasamentos['b2'].eval(session = q_network.sess))
        self.op6=self.enviasamentos['out'].assign(q_network.enviasamentos['out'].eval(session = q_network.sess))
    
        self.sess.run(self.op1)
        self.sess.run(self.op2)
        self.sess.run(self.op3)
        self.sess.run(self.op4)
        self.sess.run(self.op5)
        self.sess.run(self.op6)
   

"""Parâmetros de simulação"""
dim_accao = 3 # Número de acções, 2 para o CartPole-v0
dim_input = 2 # Posição e velocidade, no CartPole-v0 são 4
dim_h = 100  # Dimensão das hidden layers, iremos usar o mesmo número para ambas
episodes = 1000  # Número de episódios
time_lim = 10000  # Número máximo de steps
problem = 'MountainCar-v0'
# problem = 'CartPole-v0'
gamma = 0.99  # Desconto de Rewards futuras
alpha = 0.0075  # Learning rate
target_update = 1000  # número de steps entre updates da target network
REPLAY_MEMORY = 500000 #replay memory
avgRh = []  # Vector para efectuar a média de Rw
avVfh = []  # Vector para efectuar a média de Vfh
Qh = []  # Vector para armazenar Q-values de estado inicial
Vfh = [] # Vector para armazenar Values do estado final
Rw = deque([], 100)  # Últimas 100 rewards acumuladas
Vf = deque([], 100)  # Últimos 100 Values no estado terminal
render = False  # Render da simulação
render_last = 2  # Render dos últimos steps da simulação
batch_size = 8  # Size dos batchs
epsilon = 0.5  # Epsilon inicial
eps_dec = 0.995 # Taxa decrescente de epsilon
eps_min = 0.005 # Epsilon mínimo
rdeq = deque([], 100) # Rewards acumuladas por step
max_costs_hist = []
min_costs_hist = []
med_costs_hist = []

""" Simulação"""
env = gym.make(problem)
Q=Q_agente_DNN(dim_input,dim_h,dim_h,dim_accao,gamma,0.5,0.0075)
T=Target_Q_agente_DNN(dim_input,dim_h,dim_h,dim_accao,gamma,0.5,0.0075)
Q.Q_agente_initializar()
T.Q_agente_initializar()
D = deque()
tr = threading.Thread(target=toggle_rendering)
runflag = True
tr.start()




[2016-10-15 04:17:48,044] Making new env: MountainCar-v0


Q agente iniciado
Q agente iniciado


In [ ]:
try:
    for e in xrange(episodes):
        
        
        #if e==1000 or e==1500 or e==1800 or e==2500 or e==2800:
        #    print "gravar"
        #    env.monitor.start('/tmp/MountainCar-v0-'+str(e))
        
        o = env.reset() 
        oprev = np.array(o, dtype='float32')
        qi = Q.pred.eval(session=Q.sess,feed_dict = {Q.x : [oprev]})
        a = Q.actua([oprev])
        a_t = np.zeros([dim_accao])
        a_t[0] = 1
        sumR = 0
        costs_hist = []
       
            
        for t in xrange(time_lim):

            if not runflag:
                break

            if render or episodes-e < render_last:
                env.render()

            (o, r, done, info) = env.step(a)
            onext = np.array(o, dtype='float32')
            
            if done or t==time_lim-1:
                of = oprev
            rdeq.append(r)

            D.append((oprev,a_t,onext,r,done))
            
            if len(D) > REPLAY_MEMORY:
                D.popleft() 
            
            if (t+1) % target_update == 0:
                T.copiar_pesos(Q)
            
            a_t = np.zeros([dim_accao])

            
            """ Treinar a Qnet """
            if len(D) >= batch_size:
                # Sample de batches
                batch = random.sample(D, batch_size)

                s_t_batch = [d[0] for d in batch]
                a_batch = [d[1] for d in batch]
                s_t_1_batch = [d[2] for d in batch]
                r_batch = [d[3] for d in batch]
                
                # Preparar targets com a target Q-Network
                t_batch = []
                target_t1_batch = T.pred.eval(session=T.sess,feed_dict = {T.x : s_t_1_batch})
                for i in range(0, len(batch)):
                    if batch[i][4]:
                        t_batch.append(r_batch[i])
                    else:
                        t_batch.append(r_batch[i] + gamma * np.max(target_t1_batch[i]))
                
               
                costs_hist.append(Q.sess.run(Q.cost,feed_dict = {
                    Q.future_rewards : t_batch,
                    Q.a : a_batch,
                    Q.x : s_t_batch}))
                
                # Apenas um step 
                Q.sess.run(Q.optm,feed_dict = {
                    Q.future_rewards : t_batch,
                    Q.a : a_batch,
                    Q.x : s_t_batch})
                

                if random.random() <= epsilon:
                    a = random.randrange(dim_accao)
                    a_t[a] = 1
                else:
                    a = Q.actua([onext])
                    a_t[a] = 1
                    
            else:
                a = np.random.choice(dim_accao)
                a_t[a] = 1
            sumR += r
            oprev = onext
            if done:
                break
        Rw.append(sumR)
        avgRh.append(np.mean(Rw))
        Vf.append(np.max(Q.pred.eval(session=Q.sess,feed_dict = {Q.x : [of]})))
        avVfh.append(np.mean(Vf))
        Qh.append(qi[0])
        epsilon = epsilon*eps_dec if epsilon > eps_min else eps_min
        max_costs_hist.append(np.max(costs_hist))
        min_costs_hist.append(np.min(costs_hist))
        med_costs_hist.append(np.median(costs_hist))
        #if e==1000 or e==1500 or e==1800 or e==2500 or e==2800:
        #    env.monitor.close()
        print "=== Episódio %d Completo ===" %e
        print "Avg r: ", np.mean(Rw)
        print "Instant sumR: ", sumR

        if not runflag:
            break
    
    print "=== Simulação Completa ==="
    print "Score Final: %d" % avgRh[-1]

    """ Visualization of the results """
    f1 = plt.figure()
    spy = f1.add_subplot(111)
    spy.plot(avgRh,'g',label='Avg. R over 100 eps.', linewidth=2.0)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.xlabel('Episodes')
    plt.ylabel('Average Acc. R over last 100 eps')
    
    f2 = plt.figure()
    spy = f2.add_subplot(111)
    Qhnp = np.asmatrix(Qh)
    spy.plot(Qhnp[:, 0], 'b', label='Q(s0, left)', linewidth=2.0)
    spy.plot(Qhnp[:, 1], 'm', label='Q(s0, none)', linewidth=2.0)
    spy.plot(Qhnp[:, 2], 'k', label='Q(s0, right)', linewidth=2.0)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.xlabel('Episodes')
    plt.ylabel('Q-values')
    
    f3 = plt.figure()
    spy = f3.add_subplot(111)
    spy.plot(max_costs_hist, 'r', label='max cost', linewidth=2.0)
    spy.plot(min_costs_hist, 'k', label='min cost', linewidth=2.0)
    spy.plot(med_costs_hist, 'm', label='med cost', linewidth=2.0)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.xlabel('Episodes')
    plt.ylabel('Cost Function')
    
    f4 = plt.figure()
    spy = f4.add_subplot(111)
    spy.plot(avVfh, label='Value at final state', linewidth=2.0)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.xlabel('Episodes')
    plt.ylabel('Average (last 100) Values at final state')
    
    # Fechar sessões
    #Q.sess.close()
    #T.sess.close()
    
    plt.show()
except KeyboardInterrupt:
    runflag = False
    tr.join()


=== Episódio 0 Completo ===
Avg r:  -3305.0
Instant sumR:  -3305.0
=== Episódio 1 Completo ===
Avg r:  -1949.5
Instant sumR:  -594.0
=== Episódio 2 Completo ===
Avg r:  -2016.0
Instant sumR:  -2149.0
=== Episódio 3 Completo ===
Avg r:  -4012.0
Instant sumR:  -10000.0


In [24]:
var_grad = tf.gradients(Q.cost,Q.y)
var_grad2 = tf.gradients(Q.cost,Q.pred)

In [28]:
b = Q.sess.run(var_grad,feed_dict = {
                    Q.future_rewards : t_batch,
                    Q.a : a_batch,
                    Q.x : s_t_batch})
c= Q.sess.run(var_grad2,feed_dict = {
                    Q.future_rewards : t_batch,
                    Q.a : a_batch,
                    Q.x : s_t_batch})

In [30]:
b

[array([ 0.00175682, -0.00113903,  0.00285447,  0.00149781,  0.00014873,
        -0.00339086, -0.00207981,  0.00334351], dtype=float32)]

In [32]:
c

[array([[ 0.00175682,  0.        ,  0.        ],
        [-0.        , -0.        , -0.00113903],
        [ 0.00285447,  0.        ,  0.        ],
        [ 0.00149781,  0.        ,  0.        ],
        [ 0.        ,  0.00014873,  0.        ],
        [-0.00339086, -0.        , -0.        ],
        [-0.        , -0.00207981, -0.        ],
        [ 0.00334351,  0.        ,  0.        ]], dtype=float32)]

In [34]:
a_batch

[array([ 1.,  0.,  0.]),
 array([ 0.,  0.,  1.]),
 array([ 1.,  0.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 1.,  0.,  0.]),
 array([ 0.,  1.,  0.]),
 array([ 1.,  0.,  0.])]

In [54]:
cost=Q.sess.run(Q.cost,feed_dict = {
                    Q.future_rewards : t_batch,
                    Q.a : a_batch,
                    Q.x : s_t_batch})
print cost

8.35999e-05


In [38]:
yy= Q.sess.run(Q.y,feed_dict = {
                    Q.future_rewards : t_batch,
                    Q.a : a_batch,
                    Q.x : s_t_batch})

In [40]:
print yy

[-0.85418957 -0.8674137  -0.84828264 -0.85512346 -0.85979676 -0.87846535
 -0.86971986 -0.84609586]


In [42]:
print t_batch

[-0.86121686622500415, -0.86285760030150416, -0.8597005075216293, -0.86111470758914943, -0.86039168998599047, -0.86490190893411634, -0.8614006337523461, -0.85946987241506578]


In [48]:
aux=t_batch-yy
print aux

[-0.00702729  0.0045561  -0.01141787 -0.00599125 -0.00059493  0.01356345
  0.00831923 -0.01337401]
